In [13]:
from mbdvv import app, get_solids, get_s22_set, get_s66_set, kcal, ev
from pymbd import MBDCalc, from_volumes, ang, vdw_params, get_kgrid

from scipy.special import erf
import numpy as np
import pandas as pd
import os
from collections import OrderedDict
from itertools import product, islice
from functools import partial
from pkg_resources import resource_stream
from tqdm import tqdm
import re

from matplotlib import pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [4]:
def last(obj):
    if not isinstance(obj, list):
        return obj
    assert len(obj) == 2
    return obj[-1]

def listify(obj):
    if isinstance(obj, list):
        return obj
    return [obj]

def chunks(iterable, n):
    iterable = iter(iterable)
    while True:
        chunk = list(islice(iterable, n))
        if not chunk:
            break
        yield chunk
        
class Key:
    __slots__ = ['_tpl']
    
    def __init__(self, *tpl):
        self._tpl = tpl[0] if len(tpl) == 1 and isinstance(tpl, tuple) else tpl
    
    def __hash__(self):
        return hash(self._tpl)
    
    def __getitem__(self, key):
        return self._tpl[key]
    
    def __repr__(self):
        return 'Key' + repr(self._tpl)
    
    def __str__(self):
        return str(self._tpl)
    
    def __lt__(self, other):
        return self._tpl < other
    
    def __gt__(self, other):
        return self._tpl > other  
    
    def __eq__(self, other):
        return self._tpl == other

In [17]:
def ene_int(x, ds, get_key):
    key = get_key(x)
    enes = x.reset_index('key', drop=True).ene.unstack('fragment')
    cluster = ds.clusters[key]
    enes_int = cluster.get_int_ene(enes)
    return enes_int

def ref_delta(x, ds, get_key):
    ene = x.iloc[0]
    ref = ds.clusters[get_key(x)].energies['ref']
    delta = ene-ref
    reldelta = delta/abs(ref)
    return pd.Series(OrderedDict({
        'ene': ene,
        'delta': ene-ref,
        'reldelta': (ene-ref)/abs(ref),
    }))

def ene_dft_vdw(x):
    ipbe = x.index == 'PBE'
    x = x.where(ipbe, lambda y: y + x['PBE'])
    x.index = x.index.where(ipbe, 'PBE+' + x.index)
    return x
    
def ds_stat(x):
    return pd.Series(OrderedDict({
        'N': len(x.dropna()),
        'MRE': x['reldelta'].mean(),
        'MARE': abs(x['reldelta']).mean(),
        'MdRE': x['reldelta'].median(),
        'MdARE': abs(x['reldelta']).median(),
        'SDRE': x['reldelta'].std(),
        'ME': x['delta'].mean(),
        'MAE': abs(x['delta']).mean(),
    }))

def splice_key(df, indexes):
    return df.reset_index().assign(
        label=lambda x: x.key.map(lambda y: y[0]),
        scale=lambda x: x.key.map(lambda y: y[1]),
    ).drop('key', 1).set_index(['label', 'scale', *indexes])

In [6]:
class MBDException(Exception):
    pass


class NegativeEigs(MBDException):
    pass


class NegativeAlpha(MBDException):
    pass


def scaled_eigs(x):
    return np.where(x >= 0, x, -erf(np.sqrt(np.pi)/2*x**4)**(1/4))


def mbd_rsscs(mbd_calc, coords, alpha_0, C6, R_vdw, beta, lattice=None,
              k_grid=None, rpa=False, noscs=False, scale_eigs=True):
    def _array(obj, *args, **kwargs):
        if obj is not None:
            return np.array(obj, *args, **kwargs)

    coords = _array(coords, dtype=float, order='F')
    alpha_0 = _array(alpha_0, dtype=float)
    C6 = _array(C6, dtype=float)
    R_vdw = _array(R_vdw, dtype=float)
    freq, freq_w = mbd_calc.omega_grid
    omega = 4./3*C6/alpha_0**2
    if noscs:
        alpha_0_rsscs = alpha_0
        C6_rsscs = C6
        R_vdw_rsscs = R_vdw
        omega_rsscs = omega
    else:
        alpha_dyn = alpha_0/(1+(freq[:, None]/omega)**2)
        alpha_dyn_rsscs = np.empty_like(alpha_dyn)
        for a, a_scr in zip(alpha_dyn, alpha_dyn_rsscs):
            sigma = (np.sqrt(2./np.pi)*a/3)**(1./3)
            a_nlc = np.linalg.inv(
                np.diag(np.repeat(1./a, 3)) + mbd_calc.dipole_matrix(
                    coords, 'fermi,dip,gg', sigma=sigma, R_vdw=R_vdw,
                    beta=beta, lattice=lattice,
                )
            )
            a_scr[:] = np.array([a_nlc[i::3, i::3].sum(1) for i in range(3)]).sum(0)/3
        alpha_0_rsscs = alpha_dyn_rsscs[0, :]
        if np.any(alpha_0_rsscs <= 0):
            raise NegativeAlpha(alpha_0_rsscs)
        C6_rsscs = 3./np.pi*np.sum(freq_w[:, None]*alpha_dyn_rsscs**2, 0)
        R_vdw_rsscs = R_vdw*(alpha_0_rsscs/alpha_0)**(1./3)
        omega_rsscs = 4./3*C6_rsscs/alpha_0_rsscs**2
    pre = np.repeat(omega_rsscs*np.sqrt(alpha_0_rsscs), 3)
    if lattice is None:
        k_grid = [None]
    else:
        assert k_grid is not None
        k_grid = get_kgrid(lattice, k_grid)
    ene = 0
    for k_point in k_grid:
        T = mbd_calc.dipole_matrix(
            coords, 'fermi,dip', R_vdw=R_vdw_rsscs, beta=beta,
            lattice=lattice, k_point=k_point
        )
        if rpa:
            for u, uw in zip(freq[1:], freq_w[1:]):
                A = np.diag(np.repeat(alpha_0_rsscs/(1+(u/omega_rsscs)**2), 3))
                eigs = np.linalg.eigvals(A@T)
                eigs = np.real(eigs)
                if scale_eigs:
                    eigs = scaled_eigs(eigs)
                if np.any(eigs <= -1):
                    raise NegativeEigs(k_point, u, eigs)
                if not scale_eigs:
                    log_eigs = np.log(1+eigs)
                else:
                    log_eigs = np.log(1+eigs)-eigs
                ene += 1/(2*np.pi)*np.sum(log_eigs)*uw
        else:
            eigs = np.linalg.eigvalsh(
                np.diag(np.repeat(omega_rsscs**2, 3))+np.outer(pre, pre)*T
            )
            if np.any(eigs < 0):
                raise NegativeEigs(k_point, eigs)
            ene += np.sum(np.sqrt(eigs))/2-3*np.sum(omega_rsscs)/2
    ene /= len(k_grid)
    return ene

In [7]:
def mbd_from_data(calc, data, beta, vv_scale=None, vv_pol=False,
                  vv_corr=True, vdw17=False, **kwargs):
    coords = data['coords']['value'].T
    species = listify(data['elems']['atom'])
    lattice = data['lattice_vector']['value'] if 'lattice_vector' in data else None
    volumes = last(data['volumes'])
    alpha_vv = last(data['vv_pols']).copy()
    free_atoms = last(data['free_atoms'])
    species_idx = free_atoms['species']-1
    volumes_free = free_atoms['volumes'][species_idx]
    alpha_vv_free = free_atoms['vv_pols'][:, species_idx]
    freq_w = last(data['omega_grid_w'])
    C6_vv = 3/np.pi*np.sum(freq_w[:, None]*alpha_vv**2, 0)
    C6_vv_free = 3/np.pi*np.sum(freq_w[:, None]*alpha_vv_free**2, 0)
    alpha_0_free = np.array([vdw_params.get(sp)['alpha_0'] for sp in species])
    C6_free = np.array([vdw_params.get(sp)['C6'] for sp in species])

    if not vv_scale:
        volume_scale = volumes/volumes_free
    else:
        volume_scale = (alpha_vv[0]/alpha_vv_free[0])**(1/vv_scale)
    alpha_0, C6, R_vdw = from_volumes(species, volume_scale)
    if vv_pol:
        alpha_0 = alpha_vv[0]
        C6 = C6_vv
        if vv_corr:
            alpha_0 *= alpha_0_free/alpha_vv_free[0]
            C6 *= C6_free/C6_vv_free
    if vdw17:
        R_vdw = 2.5*alpha_0**(1/7)
    return mbd_rsscs(
        calc,
        coords,
        alpha_0, C6, R_vdw,
        beta,
        lattice=lattice,
        **kwargs
    )

In [8]:
def all_mbd_variants(calc, data, variants):
    k_grid = np.repeat(4, 3) if 'lattice_vector' in data else None
    enes = {}
    for label, kwargs in variants.items():
        kwargs = kwargs.copy()
        beta = kwargs.pop('beta', 0.83)
        throw = kwargs.pop('throw', False)
        try:
            ene = mbd_from_data(calc, data, beta, k_grid=k_grid, **kwargs)
        except MBDException as e:
            if throw:
                raise e
            ene = np.nan
        enes[label] = ene
    return enes

In [9]:
def calculate_solids(variants):
    df_dft, ds = get_solids(app.ctx)
    atom_enes = df_dft['atoms'].unstack().min(1).to_frame('ene').reset_index()[['atom', 'ene']].set_index('atom').ene
    df = []
    with MBDCalc() as mbd_calc:
        for (_, label, scale), data in tqdm(list(df_dft['solids'].loc(0)['solids', :, 1.].itertuples())):
            key = Key(label, scale)
            atoms = [
                ''.join(c) for c in
                chunks(re.split(r'([A-Z])', label)[1:], 2)
            ]
            pbe_ene = data['energy'][0]['value'][0]
            df.append((key, 'solid', 'PBE', pbe_ene))
            for atom in atoms:
                df.append((key, atom, 'PBE', atom_enes[atom]))
            try:
                enes = all_mbd_variants(mbd_calc, data, variants)
            except MBDException as e:
                print(label, scale, repr(e))
                continue
            for mbd_label, ene in enes.items():
                df.append((key, 'solid', mbd_label, ene))
    df = pd.DataFrame(df, columns='key fragment method ene'.split()) \
        .set_index('key fragment method'.split())
    return df, ds

def analyse_solids(df, ds):
    return (
        df.loc(0)[:, :, 'PBE']
        .groupby('key').apply(ene_int, dataset, lambda x: x.iloc[0].name[0]).stack().to_frame('ene')
        .pipe(lambda df: pd.concat((
            df,
            df.xs('PBE', level='method').join(
                dataframe.query('method != "PBE"')
                .xs('solid', level='fragment')
                .reset_index('method'),
                lsuffix='_pbe', rsuffix='_vdw'
            ).apply(lambda x: pd.Series({
                'ene': x.ene_pbe+x.ene_vdw,
                'method': 'PBE+' + str(x.method)
            }), 1).set_index('method', append=True),
        )).sort_index())
        .assign(ene=lambda x: x.ene*ev)
        .apply(ref_delta, axis=1, args=(dataset, lambda x: x.name[0]))
        .pipe(splice_key, ['method'])
        .groupby('method').apply(ds_stat)
    )

In [18]:
variants = {
    'MBD': {},
    'MBD(RPA)': {'rpa': True},
    'MBD(vdw17)': {'vdw17': True},
    'MBD(VV-scale[1])': {'vv_scale': 1},
    'MBD(vvpol)': {'vv_pol': True},
    'MBD(RPA,vvpol)': {'rpa': True, 'vv_pol': True},
    'MBD(vvpol,noscs)': {'vv_pol': True, 'noscs': True},
    'MBD(RPA,vvpol,noscs)': {'rpa': True, 'vv_pol': True, 'noscs': True},
    'MBD(vvpol,nocorr)': {'vv_pol': True, 'vv_corr': False},
    'MBD(RPA,vvpol,nocorr)': {'rpa': True, 'vv_pol': True, 'vv_corr': False},
    'MBD(vvpol,nocorr,noscs)': {'vv_pol': True, 'vv_corr': False, 'noscs': True},
    'MBD(RPA,vvpol,nocorr,noscs)': {'rpa': True, 'vv_pol': True, 'vv_corr': False, 'noscs': True},
    'MBD(vvpol,vdw17)': {'vv_pol': True, 'vdw17': True},
    'MBD(vvpol,nocorr,vdw17)': {'vv_pol': True, 'vv_corr': False, 'vdw17': True},
    'MBD(RPA,vvpol,nocorr,vdw17)': {'rpa': True, 'vv_pol': True, 'vv_corr': False, 'vdw17': True},
    'MBD(RPA,vvpol,nocorr,vdw17,noscs)': {'rpa': True, 'vv_pol': True, 'vv_corr': False, 'vdw17': True, 'noscs': True},
}
dataframe, dataset = calculate_solids(variants)
analyse_solids(dataframe, dataset)

100%|██████████| 63/63 [02:08<00:00,  2.04s/it]


,N,MRE,MARE,MdRE,MdARE,SDRE,ME,MAE
method,,,,,,,,
PBE,63.0,0.006831,0.058313,0.018849,0.047948,0.091229,0.007348,0.234291
PBE+MBD,33.0,-0.141639,0.146245,-0.121127,0.121127,0.123859,-0.560850,0.578398
PBE+MBD(RPA),52.0,-0.196318,0.199298,-0.132946,0.132946,0.208487,-0.734334,0.745688
"PBE+MBD(RPA,vvpol)",63.0,-0.161085,0.163170,-0.094389,0.094389,0.246867,-0.477931,0.485874
"PBE+MBD(RPA,vvpol,nocorr)",63.0,-0.084725,0.092305,-0.069586,0.071151,0.095333,-0.315881,0.350277
"PBE+MBD(RPA,vvpol,nocorr,noscs)",63.0,-0.087916,0.095363,-0.075088,0.076792,0.095363,-0.328180,0.361704
"PBE+MBD(RPA,vvpol,nocorr,vdw17)",63.0,-0.084784,0.091568,-0.076318,0.077226,0.092498,-0.342402,0.371043
"PBE+MBD(RPA,vvpol,nocorr,vdw17,noscs)",63.0,-0.087945,0.094667,-0.081940,0.084601,0.092492,-0.354774,0.383095
"PBE+MBD(RPA,vvpol,noscs)",63.0,-0.194436,0.194966,-0.116759,0.116759,0.277445,-0.586494,0.588514


In [19]:
def calculate_s66(variants):
    df_dft, ds = get_s66_set(app.ctx)
    df = []
    with MBDCalc(4) as mbd_calc:
        for (key, fragment), data in tqdm(list(df_dft.loc(0)[ds.name].itertuples())):
            key = Key(key)
            pbe_ene = data['energy'][0]['value'][0]
            df.append((key, fragment, 'PBE', pbe_ene))
            enes = all_mbd_variants(mbd_calc, data, variants)
            for mbd_label, ene in enes.items():
                df.append((key, fragment, mbd_label, ene))
    df = pd.DataFrame(df, columns='key fragment method ene'.split()) \
        .set_index('key fragment method'.split())
    return df, ds

def analyse_s66(df, ds):
    df = (
        df
        .groupby('key').apply(ene_int, ds, lambda x: x.iloc[0].name[0])
        .apply(ene_dft_vdw, 1).rename_axis('method', 1).stack().to_frame('ene')
        .assign(ene=lambda x: x.ene*kcal)
        .apply(ref_delta, 1, args=(ds, lambda x: x.name[0]))
        .pipe(splice_key, ['method'])
    )
    return pd.concat((
        df.loc(0)[:, 1.].groupby('method scale'.split()).apply(ds_stat),
        df.loc(0)[:, 2.].groupby('method scale'.split()).apply(ds_stat),
        df.groupby('method').apply(ds_stat).assign(scale=np.inf).set_index('scale', append=True),
    )).sort_index()

In [20]:
variants = {
    'MBD': {},
    'MBD(RPA)': {'rpa': True},
    'MBD(VV-scale[1])': {'vv_scale': 1},
    'MBD(VV-scale[1.33])': {'vv_scale': 1.33},
    'MBD(vdw17)': {'vdw17': True},
    'MBD(vdw17,beta=0.8)': {'vdw17': True, 'beta': 0.8},
    'MBD(vdw17,beta=0.86)': {'vdw17': True, 'beta': 0.86},
    'MBD(vvpol)': {'vv_pol': True},
    'MBD(vvpol,noscs)': {'vv_pol': True, 'noscs': True},
    'MBD(vvpol,nocorr)': {'vv_pol': True, 'vv_corr': False},
    'MBD(vvpol,nocorr,noscs)': {'vv_pol': True, 'vv_corr': False, 'noscs': True},
    'MBD(vvpol,noscs,vdw17)': {'vv_pol': True, 'vdw17': True, 'noscs': True},
    'MBD(vvpol,noscs,nocorr,vdw17)': {'vv_pol': True, 'vv_corr': False, 'vdw17': True, 'noscs': True},
}
dataframe, dataset = calculate_s66(variants)
analyse_s66(dataframe, dataset)

100%|██████████| 1584/1584 [00:38<00:00, 41.66it/s]


N       MRE      MARE  \
method                            scale                                 
PBE                               1.000000   66.0  0.571993  0.573934   
                                  2.000000   66.0  0.479667  0.479667   
                                  inf       528.0  0.654223  0.655971   
PBE+MBD                           1.000000   66.0 -0.058413  0.091427   
                                  2.000000   66.0 -0.077591  0.080452   
                                  inf       528.0 -0.063305  0.114756   
PBE+MBD(RPA)                      1.000000   66.0 -0.060806  0.092571   
                                  2.000000   66.0 -0.080211  0.082946   
                                  inf       528.0 -0.066118  0.115898   
PBE+MBD(VV-scale[1.33])           1.000000   66.0 -0.035437  0.090784   
                                  2.000000   66.0 -0.145850  0.150462   
                                  inf       528.0 -0.052339  0.131171   
PBE+MBD(VV-scale[1])              1.000000   66.0  0.017968  0.095332   
                                  2.000000   66.0 -0.114720  0.131818   
                                  inf       528.0  0.005933  0.131097   
PBE+MBD(vdw17)                    1.000000   66.0 -0.030194  0.081623   
                                  2.000000   66.0 -0.087136  0.089928   
                                  inf       528.0 -0.039378  0.116329   
PBE+MBD(vdw17,beta=0.8)           1.000000   66.0 -0.076798  0.097198   
                                  2.000000   66.0 -0.084362  0.087158   
                                  inf       528.0 -0.083898  0.116431   
PBE+MBD(vdw17,beta=0.86)          1.000000   66.0  0.017333  0.088929   
                                  2.000000   66.0 -0.089925  0.092713   
                                  inf       528.0  0.005498  0.129223   
PBE+MBD(vvpol)                    1.000000   66.0 -0.001073  0.078185   
                                  2.000000   66.0 -0.075146  0.090129   
                                  inf       528.0 -0.007815  0.113626   
PBE+MBD(vvpol,nocorr)             1.000000   66.0 -0.125801  0.129342   
                                  2.000000   66.0 -0.153905  0.166671   
                                  inf       528.0 -0.142762  0.159607   
PBE+MBD(vvpol,nocorr,noscs)       1.000000   66.0 -0.138726  0.147958   
                                  2.000000   66.0 -0.157935  0.159156   
                                  inf       528.0 -0.154144  0.181083   
PBE+MBD(vvpol,noscs)              1.000000   66.0  0.009308  0.096371   
                                  2.000000   66.0 -0.042202  0.055091   
                                  inf       528.0  0.009519  0.128320   
PBE+MBD(vvpol,noscs,nocorr,vdw17) 1.000000   66.0 -0.036228  0.087582   
                                  2.000000   66.0  0.164274  0.185716   
                                  inf       528.0 -0.001177  0.124471   
PBE+MBD(vvpol,noscs,vdw17)        1.000000   66.0  0.074751  0.120540   
                                  2.000000   66.0  0.220913  0.229383   
                                  inf       528.0  0.118683  0.162882   

                                                MdRE     MdARE      SDRE  \
method                            scale                                    
PBE                               1.000000  0.548833  0.548833  0.459117   
                                  2.000000  0.290758  0.290758  0.738922   
                                  inf       0.428027  0.428027  1.509132   
PBE+MBD                           1.000000 -0.049618  0.076535  0.104359   
                                  2.000000 -0.045896  0.046321  0.150255   
                                  inf      -0.068312  0.079864  0.203219   
PBE+MBD(RPA)                      1.000000 -0.050783  0.078433  0.104912   
                                  2.000000 -0.046507  0.046637  0.155160   
                                  inf      -0.069181  0.081062  0.200622   